In [1]:
import os
import pickle
from win32gui import FindWindow, SetForegroundWindow, GetClientRect, ClientToScreen
import pyautogui
import cv2
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk
from matplotlib import pyplot as plt
from ctypes import windll
from math import floor, ceil
import win32api
import time
from threading import Thread
import tensorflow as tf

In [2]:
def Coords(Pred, small):
	#cv2, Predicitons.pickle[2]=small
	Pred = (255 - cv2.inRange(np.array(Pred * 255, dtype = np.uint8), 0, 160)).astype(np.float32)
	small = np.array(small * 255, dtype = np.uint8).astype(np.float32)
	result = cv2.matchTemplate(Pred, small, cv2.TM_SQDIFF_NORMED)
	mn,_,mnLoc,_ = cv2.minMaxLoc(result)
	MPx,MPy = mnLoc
	#x and y may flipped
	return (MPx+2-256, MPy+2-256)
	#return (MPx, MPy)

In [3]:
def ImCompareGray(Im1, Im2, FigSize=(10,10)):
    f = plt.figure(figsize=FigSize)
    f.add_subplot(1,2, 1)
    plt.imshow(Im1, "gray")
    f.add_subplot(1,2, 2)
    plt.imshow(Im2)
    plt.show(block=True)

In [4]:
checkpoint_path = "C:/Users/8holz/Dokumente/GitHub/K14/Training_Protocol/Model-Color-UNET-Terrosristhunt/weights/saved_model.pb"
DATADIR5 = r"D:\K14\Dataset\Predicitions.pickle"
checkpoint_dir = os.path.dirname(checkpoint_path)

WindowClassName0 = 'Rainbow Six'
WindowClassName1 = 'J5 - Jupyter Notebook - Google Chrome'
WindowClassName2 = 'tk'
WindowClassName3 = 'Fotos'

In [5]:
def load_K14():
    #model = create_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
    #model.summary()
    model = tf.keras.models.load_model(checkpoint_dir)
    #model.summary()

    configproto = tf.compat.v1.ConfigProto() 
    configproto.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=configproto) 
    tf.compat.v1.keras.backend.set_session(sess)
    return model

In [6]:
model = load_K14()

In [7]:
def xyz(image):
    a = time.time()
    im_preds = model.predict_on_batch(image)
    #print(time.time()-a)
    image = image.reshape(512, 512, 3)
    im_preds = tf.reshape(im_preds, [512, 512, 1]).numpy()
    #ImCompareGray(image, im_preds)
    return im_preds

In [8]:
def screenshot(window_title=None, lenx=512, leny=512):
	im = pyautogui.screenshot(region=(1024, 464, 512, 512))
	gray = np.array(im)
	image = np.array(gray).reshape(-1, lenx, leny, 3)
	return image

In [9]:
xyz(screenshot())

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node model_4/conv2d_76/StatefulPartitionedCall/StatefulPartitionedCall/Conv2D}}]] [Op:__inference_function_7176]

Function call stack:
function


In [ ]:
class myThread(Thread):
	def __init__(self):
		Thread.__init__(self)
		None
	def run(self):
		# saves the images in the global list
		root = tk.Tk()
		canvas0 = tk.Canvas(root, width = 512, height = 512)
		canvas0.pack()
		canvas1 = tk.Canvas(root, width = 512, height = 512)
		canvas1.pack()
		def recursive():
			global ImageTesten0, ImageTesten1, PredImage0, PredImage1
			ImageTesten0 = ImageTk.PhotoImage(Image.fromarray((cv2.cvtColor(PredImage0,cv2.COLOR_GRAY2RGB) * 255).astype(np.uint8)))
			ImageTesten1 = ImageTk.PhotoImage(Image.fromarray(cv2.cvtColor(PredImage1, cv2.COLOR_BGR2RGB)))
			#ImageTesten = tk.PhotoImage(file=DATADIR2+random.choice(os.listdir(DATADIR2)))
			canvas0.create_image(0,0, anchor=tk.NW, image=ImageTesten0)
			canvas1.create_image(0,0, anchor=tk.NW, image=ImageTesten1)
			root.after(100, recursive)
		recursive()
		root.mainloop()

In [ ]:
Template = pickle.load(open(r"D:\K14\Dataset\Predicitions.pickle", "rb"))[10]
ims = pickle.load(open(r"D:\K14\Dataset\TestImages.pickle", "rb"))
Cache = pickle.load(open(r"C:\Users\8holz\Dokumente\GitHub\K14\Cache.pickle", "rb"))
Im0 = ims[1][8]

In [ ]:
PredImage0 = None
ImageTesten0 = None
PredImage1 = None
ImageTesten1 = None

In [ ]:
#PredImage0 = cv2.rectangle(PredImage, (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)
#PredImage1 = cv2.rectangle(ims[0][8], (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)
PredImage0 = ims[1][8]
PredImage1 = ims[0][1]
print(PredImage1.shape)

In [ ]:
inst = myThread()
inst.daemon = True

In [ ]:
inst.start()

In [ ]:
c = Coords(PredImage0, Template)

In [ ]:
PredImage0 = cv2.rectangle(PredImage0, (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)
PredImage1 = cv2.rectangle(ims[0][8], (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)

In [ ]:
time.sleep(5)

In [ ]:
while True:
    ScrSh = screenshot(WindowClassName3)
    Mask = xyz(ScrSh)
    c = Coords(Mask, Template)
    PredImage0 = cv2.rectangle(Mask, (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)
    PredImage1 = cv2.rectangle(ScrSh.reshape(512,512,3), (c[0]+256, c[1]+256), (c[0]+256+2, c[1]+256+2), (0,255,0), 2)
    time.sleep(0.005)